# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,11.24,76,0,1.69,AU,1726520149
1,1,olonkinbyen,70.9221,-8.7187,0.80,79,100,4.56,SJ,1726520151
2,2,tamanrasset,22.7850,5.5228,24.95,57,40,2.06,DZ,1726520188
3,3,udachny,66.4167,112.4000,3.06,97,100,8.45,RU,1726520165
4,4,adamstown,-25.0660,-130.1015,19.27,85,100,9.83,PN,1726520146


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    color = "City",
    tiles = "OSM",  
    frame_width = 700,
    frame_height = 500   
)
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
cleaned_df = filtered_df.dropna(subset=['City', 'Lng', 'Lat', 'Humidity'])

# Display sample data
df_for_cleaned_df = pd.DataFrame(cleaned_df)
df_for_cleaned_df.sample(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
364,364,alexandria,31.2156,29.9553,26.15,58,0,3.40,EG,1726520400
509,509,likasi,-10.9814,26.7333,20.74,26,0,1.51,CD,1726520783
0,0,margaret river,-33.9500,115.0667,11.24,76,0,1.69,AU,1726520149
351,351,vadso,70.0744,29.7487,8.09,87,0,1.03,NO,1726520601
565,565,horqueta,-23.3428,-57.0597,25.19,41,0,2.20,PY,1726520844
154,154,geraldton,-28.7667,114.6000,12.23,58,0,4.12,AU,1726520154
45,45,hermanus,-34.4187,19.2345,9.33,77,0,3.86,ZA,1726520155
197,197,hammerfest,70.6634,23.6821,11.25,82,0,1.54,NO,1726520424
159,159,pasighat,28.0667,95.3333,25.37,86,0,1.45,IN,1726520376
311,311,graaff-reinet,-32.2522,24.5308,9.40,81,0,1.04,ZA,1726520553


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= cleaned_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
df_for_display = pd.DataFrame(hotel_df)
df_for_display

,City,Country,Lat,Lng,Humidity,Hotel Name
0,margaret river,AU,-33.9500,115.0667,76,
13,bethel,US,41.3712,-73.4140,53,
45,hermanus,ZA,-34.4187,19.2345,77,
49,bredasdorp,ZA,-34.5322,20.0403,86,
53,port elizabeth,ZA,-33.9180,25.5701,87,
64,nampula,MZ,-15.1165,39.2666,60,
89,turpan,CN,42.9333,89.1667,22,
90,zvishavane,ZW,-20.3267,30.0665,27,
104,stanley,GB,54.8680,-1.6985,90,
119,revda,RU,56.8010,59.9303,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
api_key = "9317a2198727463eb60c6b3bf6ebab0a"
params = {
    'categories': 'accommodation.hotel',
    'radius': radius,
    'apiKey': api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
bethel - nearest hotel: Hampton Inn Danbury
hermanus - nearest hotel: Aloe guest house
bredasdorp - nearest hotel: Victoria Hotel
port elizabeth - nearest hotel: Waterford Hotel
nampula - nearest hotel: Hotel Lurio
turpan - nearest hotel: Oriental
zvishavane - nearest hotel: Zvishavane Panorama Lodge
stanley - nearest hotel: Hotel 52
revda - nearest hotel: Металлург
castillos - nearest hotel: A mi gente
hampton bays - nearest hotel: Canoe Place
geraldton - nearest hotel: Ocean Centre Hotel
pasighat - nearest hotel: Aane Hotel
hammerfest - nearest hotel: Thon Hotel Hammerfest
alta - nearest hotel: Scandic Alta
belmonte - nearest hotel: No hotel found
primero de mayo - nearest hotel: No hotel found
chui - nearest hotel: Alerces
tolanaro - nearest hotel: Hôtel Mahavokey
broken hill - nearest hotel: Ibis Styles
graaff-reinet - nearest hotel: Profcon Resort
ashdod - nearest hotel: Spat Ashdod
kargil - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,margaret river,AU,-33.9500,115.0667,76,Margaret River Hotel
13,bethel,US,41.3712,-73.4140,53,Hampton Inn Danbury
45,hermanus,ZA,-34.4187,19.2345,77,Aloe guest house
49,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
53,port elizabeth,ZA,-33.9180,25.5701,87,Waterford Hotel
64,nampula,MZ,-15.1165,39.2666,60,Hotel Lurio
89,turpan,CN,42.9333,89.1667,22,Oriental
90,zvishavane,ZW,-20.3267,30.0665,27,Zvishavane Panorama Lodge
104,stanley,GB,54.8680,-1.6985,90,Hotel 52
119,revda,RU,56.8010,59.9303,71,Металлург


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size='Humidity',
    color = "City",
    tiles = "OSM",
    hover_cols=['Hotel Name','City', 'Country']
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)